# Natural Language Processing + Decision Science

👩🏻‍🏫 In this challenge, we will combine:
* 🗣 Natural Language Processing 
* 📊 Decision Science 

🎯 The goal is to understand the bad reviews of the products and the sellers on Olist

In [1]:
# Data Manipulation
import numpy as np
import pandas as pd
pd.set_option("display.max_columns",None)

# Olist packages
from olist.data import Olist
from olist.review import Review
from olist.order import Order
from olist.product_updated import Product
from olist.seller_updated import Seller

# Machine Learning
from sklearn.pipeline import make_pipeline

# Language Processing
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
import nltk
import string
import text_unidecode as unidecode

# Vectorizers and NLP Models
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

🕵🏻‍♂️ Imagine that the CEO of Olist [Tiago Dalvi](https://www.linkedin.com/in/tiagodalvi/) ask you to read and understand the reviews.

- What did customers say about their order if they rated it 1? 2? 3?
- What are the most frequent bad reviews about...
    - the worst rated products ?
    - the wort sellers ?

## (0) Setup

👩🏻‍🏫 If you followed the `Decision Science` module, you already have the `olist` package installed and importable. *You can skip this section* and move to the **Data collection** section.

### (0.1) Import `olist` package`

Download a fresh version of the `olist` package:

```bash
mkdir ~/code/lewagon
cd ~/code/lewagon
git clone git@github.com:lewagon/olist.git
cd olist
git fetch
git checkout full-package
```

### (0.2) Download the datasets

- Download the datasets from Kaggle https://www.kaggle.com/olistbr/brazilian-ecommerce
- Unzip them into the `/data/csv` directory of the `olist` package:

```bash
.
├── README.md
├── data
│   └── csv
│       ├── olist_customers_dataset.csv
│       ├── olist_geolocation_dataset.csv
│       ├── olist_order_items_dataset.csv
│       ├── olist_order_payments_dataset.csv
│       ├── olist_order_reviews_dataset.csv
│       ├── olist_orders_dataset.csv
│       ├── olist_products_dataset.csv
│       ├── olist_sellers_dataset.csv
│       └── product_category_name_translation.csv
├── notebooks
├── olist
│   ├── README.md
│   ├── __init__.py
│   ├── data.py
│   ├── order.py
│   ├── product.py
│   ├── product_updated.py
│   ├── review.py
│   ├── seller.py
│   ├── seller_updated.py
│   └── utils.py
├── requirements.txt
└── setup.p
```

### (0.3) Install the `olist` package

```bash
pip install -e .
```

⚠️ Restart the kernel.

## (1) Data Collection and Preparation

👉 The way we designed the `reviews` DataFrame will help us focus on the `product categories`.

In [2]:
reviews = Review().get_training_data()
reviews.head()

,review_id,length_review,review_score,order_id,product_category_name
0,7bc2406110b926393aa56f80a40eba40,0,4,73fc7af87114b39712e6da79b0a377eb,esporte_lazer
1,80e641a11e56f04c1ad469d5645fdfde,0,5,a548910a1c6147796b98fdf73dbeba33,informatica_acessorios
2,228ce5500dc1d8e020d8d1322874b6f0,0,5,f9e4b658b201a9f2ecdecbb34bed034b,informatica_acessorios
3,e64fb393e7b32834bb789ff8bb30750e,37,5,658677c97b385a9be170737859d3511b,ferramentas_jardim
4,f7c4243c7fe1938f181bec41a392bdeb,100,5,8e6bfb81e283fa7e4f11123a3fb894f1,esporte_lazer


👉 We can retrieve the reviews from `order_reviews` within the Olist class.

In [3]:
data = Olist().get_data()

In [4]:
reviews_data = data['order_reviews']
reviews_data.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


🪤 We need to collect some information about the orders because:
- Back then, customers could review an order even before receiving it
- They could also rate an order that they hadn't received

In [5]:
orders = data['orders']
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


💥 Let's merge these three datasets and apply some operations

In [6]:
# YOUR CODE HERE
df = reviews.merge(reviews_data).merge(orders)
df.shape

(98657, 16)

🚓 Remove the reviews that were written even before receiving the order.

In [7]:
# YOUR CODE HERE
condition = df["order_delivered_customer_date"] < df["review_creation_date"]
df = df[condition]
df.shape

(88005, 16)

🚓 Remove the reviews for undelivered orders.

In [8]:
# YOUR CODE HERE
condition = df["order_status"] == "delivered"
df = df[condition]
df.shape

(88000, 16)

In [9]:
df.head()

,review_id,length_review,review_score,order_id,product_category_name,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,7bc2406110b926393aa56f80a40eba40,0,4,73fc7af87114b39712e6da79b0a377eb,esporte_lazer,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59,41dcb106f807e993532d446263290104,delivered,2018-01-11 15:30:49,2018-01-11 15:47:59,2018-01-12 21:57:22,2018-01-17 18:42:41,2018-02-02 00:00:00
1,80e641a11e56f04c1ad469d5645fdfde,0,5,a548910a1c6147796b98fdf73dbeba33,informatica_acessorios,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13,8a2e7ef9053dea531e4dc76bd6d853e6,delivered,2018-02-28 12:25:19,2018-02-28 12:48:39,2018-03-02 19:08:15,2018-03-09 23:17:20,2018-03-14 00:00:00
2,228ce5500dc1d8e020d8d1322874b6f0,0,5,f9e4b658b201a9f2ecdecbb34bed034b,informatica_acessorios,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24,e226dfed6544df5b7b87a48208690feb,delivered,2018-02-03 09:56:22,2018-02-03 10:33:41,2018-02-06 16:18:28,2018-02-16 17:28:48,2018-03-09 00:00:00
3,e64fb393e7b32834bb789ff8bb30750e,37,5,658677c97b385a9be170737859d3511b,ferramentas_jardim,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06,de6dff97e5f1ba84a3cd9a3bc97df5f6,delivered,2017-04-09 17:41:13,2017-04-09 17:55:19,2017-04-10 14:24:47,2017-04-20 09:08:35,2017-05-10 00:00:00
4,f7c4243c7fe1938f181bec41a392bdeb,100,5,8e6bfb81e283fa7e4f11123a3fb894f1,esporte_lazer,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53,5986b333ca0d44534a156a52a8e33a83,delivered,2018-02-10 10:59:03,2018-02-10 15:48:21,2018-02-15 19:36:14,2018-02-28 16:33:35,2018-03-09 00:00:00


✍️ As some people fill in only the title or the body of a review, it may be a good idea to aggregate them together.



<details>
    <summary>💡<i>Hint</i></summary>

* Drop rows with both a missing title and a missing comment
* Refer to the documentation [pandas.DataFrame.dropna()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html) and have a look at the `how` argument...

In [10]:
# YOUR CODE HERE
df = df.dropna(subset=["review_comment_title", "review_comment_message"], how="all")
df.shape

(36148, 16)

In [11]:
df["review_total"] = df["review_comment_title"].fillna("") + df["review_comment_message"].fillna("")
df.head()

,review_id,length_review,review_score,order_id,product_category_name,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,review_total
3,e64fb393e7b32834bb789ff8bb30750e,37,5,658677c97b385a9be170737859d3511b,ferramentas_jardim,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06,de6dff97e5f1ba84a3cd9a3bc97df5f6,delivered,2017-04-09 17:41:13,2017-04-09 17:55:19,2017-04-10 14:24:47,2017-04-20 09:08:35,2017-05-10 00:00:00,Recebi bem antes do prazo estipulado.
4,f7c4243c7fe1938f181bec41a392bdeb,100,5,8e6bfb81e283fa7e4f11123a3fb894f1,esporte_lazer,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53,5986b333ca0d44534a156a52a8e33a83,delivered,2018-02-10 10:59:03,2018-02-10 15:48:21,2018-02-15 19:36:14,2018-02-28 16:33:35,2018-03-09 00:00:00,Parabéns lojas lannister adorei comprar pela I...
9,8670d52e15e00043ae7de4c01cc2fe06,174,4,b9bf720beb4ab3728760088589c62129,eletroportateis,recomendo,aparelho eficiente. no site a marca do aparelh...,2018-05-22 00:00:00,2018-05-23 16:45:47,a5224bdc7685fd39cd7a23404415493d,delivered,2018-05-14 10:29:02,2018-05-15 10:37:47,2018-05-15 13:29:00,2018-05-21 17:52:12,2018-06-06 00:00:00,recomendoaparelho eficiente. no site a marca d...
12,4b49719c8a200003f700d3d986ea1a19,45,4,9d6f15f95d01e79bd1349cc208361f09,beleza_saude,NaN,"Mas um pouco ,travando...pelo valor ta Boa.\r\n",2018-02-16 00:00:00,2018-02-20 10:52:22,8ebbeb09e94d1e789142983f73faa9cd,delivered,2018-02-12 16:07:34,2018-02-12 16:27:39,2018-02-14 20:04:16,2018-02-15 19:16:54,2018-02-26 00:00:00,"Mas um pouco ,travando...pelo valor ta Boa.\r\n"
15,3948b09f7c818e2d86c9a546758b2335,56,5,e51478e7e277a83743b6f9991dbfa3fb,informatica_acessorios,Super recomendo,"Vendedor confiável, produto ok e entrega antes...",2018-05-23 00:00:00,2018-05-24 03:00:01,659ded3e9b43aaf51cf9586d03033b46,delivered,2018-05-18 18:20:45,2018-05-18 18:35:28,2018-05-19 09:27:00,2018-05-22 14:58:47,2018-06-07 00:00:00,"Super recomendoVendedor confiável, produto ok ..."


🕵🏻 Let's focus on some columns of interest:

In [12]:
# Focus on certain columns
columns_of_interest = ['review_id',
                       'length_review',
                       'review_score',
                       'order_id',
                       'product_category_name',
                       'review_total']
df = df[columns_of_interest] 

In [13]:
df.head()

,review_id,length_review,review_score,order_id,product_category_name,review_total
3,e64fb393e7b32834bb789ff8bb30750e,37,5,658677c97b385a9be170737859d3511b,ferramentas_jardim,Recebi bem antes do prazo estipulado.
4,f7c4243c7fe1938f181bec41a392bdeb,100,5,8e6bfb81e283fa7e4f11123a3fb894f1,esporte_lazer,Parabéns lojas lannister adorei comprar pela I...
9,8670d52e15e00043ae7de4c01cc2fe06,174,4,b9bf720beb4ab3728760088589c62129,eletroportateis,recomendoaparelho eficiente. no site a marca d...
12,4b49719c8a200003f700d3d986ea1a19,45,4,9d6f15f95d01e79bd1349cc208361f09,beleza_saude,"Mas um pouco ,travando...pelo valor ta Boa.\r\n"
15,3948b09f7c818e2d86c9a546758b2335,56,5,e51478e7e277a83743b6f9991dbfa3fb,informatica_acessorios,"Super recomendoVendedor confiável, produto ok ..."


## (2) Text Cleaning

🧹 Create a function `cleaning(sentence)` and apply it to the reviews. **Beware NLTK has no Portuguese lemmatizer** (often but they do have a stemmer `nltk.stem.RSLPStemmer`. 

In [14]:
# YOUR CODE HERE
from nltk.stem import RSLPStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download("rslp")
stemmer = RSLPStemmer()
import unidecode

[nltk_data] Downloading package rslp to /Users/laurameyer/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [15]:
# YOUR CODE HERE
import string

def cleaning(sentence):

    ## remove whitespaces
    sentence = sentence.strip()
    
    ## lowercasing 
    sentence = sentence.lower()
    ## removing numbers
    sentence = "".join([char for char in sentence if not char.isdigit()])
    
    # remove accents
    sentence = unidecode.unidecode(sentence)
    
    ## removing punctuation
    for punctuation in string.punctuation: 
        sentence = sentence.replace(punctuation, "")

    ## tokenizing 
    tokens = word_tokenize(sentence)
    
    ## defining stopwords
    stop_words = set(stopwords.words("portuguese"))
    
    ## remove stopwords
    tokens_without_stopwords = [token for token in tokens if not token in stop_words]
    
    # lemmatizer
    return " ".join([stemmer.stem(token) for token in tokens_without_stopwords])

In [16]:
df["review_clean"] = df["review_total"].apply(cleaning)
df.head()

,review_id,length_review,review_score,order_id,product_category_name,review_total,review_clean
3,e64fb393e7b32834bb789ff8bb30750e,37,5,658677c97b385a9be170737859d3511b,ferramentas_jardim,Recebi bem antes do prazo estipulado.,receb bem ant praz estipul
4,f7c4243c7fe1938f181bec41a392bdeb,100,5,8e6bfb81e283fa7e4f11123a3fb894f1,esporte_lazer,Parabéns lojas lannister adorei comprar pela I...,parab loj lannist ador compr internet segur pr...
9,8670d52e15e00043ae7de4c01cc2fe06,174,4,b9bf720beb4ab3728760088589c62129,eletroportateis,recomendoaparelho eficiente. no site a marca d...,recomendoaparelh efici sit marc aparelh impres...
12,4b49719c8a200003f700d3d986ea1a19,45,4,9d6f15f95d01e79bd1349cc208361f09,beleza_saude,"Mas um pouco ,travando...pelo valor ta Boa.\r\n",pouc travandopel val ta boa
15,3948b09f7c818e2d86c9a546758b2335,56,5,e51478e7e277a83743b6f9991dbfa3fb,informatica_acessorios,"Super recomendoVendedor confiável, produto ok ...",sup recomendovend confia produt ok entreg ant ...


In [17]:
df.head()

,review_id,length_review,review_score,order_id,product_category_name,review_total,review_clean
3,e64fb393e7b32834bb789ff8bb30750e,37,5,658677c97b385a9be170737859d3511b,ferramentas_jardim,Recebi bem antes do prazo estipulado.,receb bem ant praz estipul
4,f7c4243c7fe1938f181bec41a392bdeb,100,5,8e6bfb81e283fa7e4f11123a3fb894f1,esporte_lazer,Parabéns lojas lannister adorei comprar pela I...,parab loj lannist ador compr internet segur pr...
9,8670d52e15e00043ae7de4c01cc2fe06,174,4,b9bf720beb4ab3728760088589c62129,eletroportateis,recomendoaparelho eficiente. no site a marca d...,recomendoaparelh efici sit marc aparelh impres...
12,4b49719c8a200003f700d3d986ea1a19,45,4,9d6f15f95d01e79bd1349cc208361f09,beleza_saude,"Mas um pouco ,travando...pelo valor ta Boa.\r\n",pouc travandopel val ta boa
15,3948b09f7c818e2d86c9a546758b2335,56,5,e51478e7e277a83743b6f9991dbfa3fb,informatica_acessorios,"Super recomendoVendedor confiável, produto ok ...",sup recomendovend confia produt ok entreg ant ...


## (3) Analysis of bad reviews

### (3.1) Dataset with low review scores

😱 What is the proportion of reviews with a rating between 1 and 3 ? 

In [18]:
# YOUR CODE HERE
df["review_score"].value_counts(normalize=True).sort_values(ascending=False)[2:5]

1    0.134447
3    0.086810
2    0.045922
Name: review_score, dtype: float64

🕵🏻‍♂️ Let's focus on these reviews...

In [19]:
# YOUR CODE HERE
condition = df["review_score"] < 4
df = df[condition]
df.shape

(9658, 7)

### (3.2) Vectorizing

🔡 ➡️ 🔢 Vectorize your texts. 

* Make sure to take into accounts bigrams.
* Set a `max_df` to $0.75$ to remove too frequent words
* Spoiler alert: you will end up with $20k+$ words... let's keep only `max_features` = $5000$ for this challenge.

In [20]:
# YOUR CODE HERE
from sklearn.feature_extraction.text import TfidfVectorizer

X = df["review_clean"]
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_df=0.75, max_features=5000)
vectorized_X = pd.DataFrame(vectorizer.fit_transform(X).toarray(), columns=vectorizer.get_feature_names_out())
vectorized_X

abaix  abert  aborrec  abr  abr caix  abr cham  abr embal  abr fech  \
0       0.0    0.0      0.0  0.0       0.0       0.0        0.0       0.0   
1       0.0    0.0      0.0  0.0       0.0       0.0        0.0       0.0   
2       0.0    0.0      0.0  0.0       0.0       0.0        0.0       0.0   
3       0.0    0.0      0.0  0.0       0.0       0.0        0.0       0.0   
4       0.0    0.0      0.0  0.0       0.0       0.0        0.0       0.0   
...     ...    ...      ...  ...       ...       ...        ...       ...   
9653    0.0    0.0      0.0  0.0       0.0       0.0        0.0       0.0   
9654    0.0    0.0      0.0  0.0       0.0       0.0        0.0       0.0   
9655    0.0    0.0      0.0  0.0       0.0       0.0        0.0       0.0   
9656    0.0    0.0      0.0  0.0       0.0       0.0        0.0       0.0   
9657    0.0    0.0      0.0  0.0       0.0       0.0        0.0       0.0   

      abr reclamaca  abril  absurd  absurd entreg      acab  acab deix  \
0               0.0    0.0     0.0            0.0  0.000000        0.0   
1               0.0    0.0     0.0            0.0  0.000000        0.0   
2               0.0    0.0     0.0            0.0  0.000000        0.0   
3               0.0    0.0     0.0            0.0  0.374766        0.0   
4               0.0    0.0     0.0            0.0  0.000000        0.0   
...             ...    ...     ...            ...       ...        ...   
9653            0.0    0.0     0.0            0.0  0.000000        0.0   
9654            0.0    0.0     0.0            0.0  0.000000        0.0   
9655            0.0    0.0     0.0            0.0  0.000000        0.0   
9656            0.0    0.0     0.0            0.0  0.000000        0.0   
9657            0.0    0.0     0.0            0.0  0.000000        0.0   

      acab fic  acab nao  acab receb  acab ruim  aceit  acend  acert  acess  \
0          0.0       0.0         0.0        0.0    0.0    0.0    0.0    0.0   
1          0.0       0.0         0.0        0.0    0.0    0.0    0.0    0.0   
2          0.0       0.0         0.0        0.0    0.0    0.0    0.0    0.0   
3          0.0       0.0         0.0        0.0    0.0    0.0    0.0    0.0   
4          0.0       0.0         0.0        0.0    0.0    0.0    0.0    0.0   
...        ...       ...         ...        ...    ...    ...    ...    ...   
9653       0.0       0.0         0.0        0.0    0.0    0.0    0.0    0.0   
9654       0.0       0.0         0.0        0.0    0.0    0.0    0.0    0.0   
9655       0.0       0.0         0.0        0.0    0.0    0.0    0.0    0.0   
9656       0.0       0.0         0.0        0.0    0.0    0.0    0.0    0.0   
9657       0.0       0.0         0.0        0.0    0.0    0.0    0.0    0.0   

      acessori  ach  ach absurd  ach bem  ach car  ach cord  ach demor  \
0          0.0  0.0         0.0      0.0      0.0       0.0        0.0   
1          0.0  0.0         0.0      0.0      0.0       0.0        0.0   
2          0.0  0.0         0.0      0.0      0.0       0.0        0.0   
3          0.0  0.0         0.0      0.0      0.0       0.0        0.0   
4          0.0  0.0         0.0      0.0      0.0       0.0        0.0   
...        ...  ...         ...      ...      ...       ...        ...   
9653       0.0  0.0         0.0      0.0      0.0       0.0        0.0   
9654       0.0  0.0         0.0      0.0      0.0       0.0        0.0   
9655       0.0  0.0         0.0      0.0      0.0       0.0        0.0   
9656       0.0  0.0         0.0      0.0      0.0       0.0        0.0   
9657       0.0  0.0         0.0      0.0      0.0       0.0        0.0   

      ach dev  ach entreg  ach estranh  ach fornec  ach fret  ach loj  \
0         0.0         0.0          0.0         0.0       0.0      0.0   
1         0.0         0.0          0.0         0.0       0.0      0.0   
2         0.0         0.0          0.0         0.0       0.0      0.0   
3         0.0         0.0          0.0         0.0       0.0  

### (3.3) LDA

🕵🏻‍♂️ Fit an LDA:
- Choose `n_components = 3`
- Show the Document Mixture of Topics  with *.transform()*
- Show the Topic Mixture with *.components_*

In [21]:
# YOUR CODE HERE
from sklearn.decomposition import LatentDirichletAllocation 

n_components = 3

lda = LatentDirichletAllocation(n_components=n_components, max_iter=100)

X_transformed = lda.fit_transform(vectorized_X)
X_transformed

array([[0.35695969, 0.56419012, 0.07885019],
       [0.84510392, 0.07607944, 0.07881664],
       [0.0719415 , 0.85520531, 0.07285319],
       ...,
       [0.21937883, 0.09021675, 0.69040442],
       [0.429392  , 0.08250047, 0.48810753],
       [0.07829167, 0.10327051, 0.81843782]])

#### Document Mixture (of Topics)

In [24]:
# YOUR CODE HERE
pd.DataFrame(X_transformed, columns=[f"Topic {i +1}" for i in range(n_components)])

,Topic 1,Topic 2,Topic 3
0,0.356960,0.564190,0.078850
1,0.845104,0.076079,0.078817
2,0.071941,0.855205,0.072853
3,0.101112,0.797729,0.101159
4,0.694906,0.176893,0.128201
...,...,...,...
9653,0.849537,0.074970,0.075493
9654,0.101995,0.791808,0.106197
9655,0.219379,0.090217,0.690404
9656,0.429392,0.082500,0.488108


👉 Let's report the most important topic for each review

In [28]:
# YOUR CODE HERE

df["Topic"] = np.argmax(X_transformed, axis=1)
df.head()

,review_id,length_review,review_score,order_id,product_category_name,review_total,review_clean,Topic
53,e233e51d11511bf30e568c76360ace52,97,1,548df2c6e5f089574614894bca78acf5,eletronicos,recebi somente 1 controle Midea Split ESTILO.\...,receb soment control mide split estil falt con...,1
75,8b230a1373c6dc4bd867099fda1d7039,60,3,071251fe3b3493294536f03737a8a679,ferramentas_jardim,Eu comprei duas unidades e só recebi uma e ago...,compr dua unidad so receb agor fac,0
77,cb2fc3e5711b5ae85e0491ee18af63ed,72,3,34e6d418f368f8079ae152bc178bc66a,beleza_saude,"Produto bom, porém o que veio para mim não con...",produt bom por vei mim nao condiz fot anunci,1
78,60c714ed14cef913944a3147094a4742,36,1,9ac05114800f02bfaa783bd76842dbe2,moveis_decoracao,"Produto muito inferior, mal acabado.",produt inferi mal acab,1
89,0bd4dcc4f6c4621baf37f73495cad8c4,16,3,a11cd01ac67beef7e8bf09740d8a35c1,esporte_lazer,Entrega no prazo,entreg praz,0


#### Topic Mixture (of Words)

In [34]:
# YOUR CODE HERE
topic_mixture = pd.DataFrame(lda.components_, columns=vectorizer.get_feature_names_out(), index=["Topic 1", "Topic 2", "Topic 3"])
topic_mixture

abaix     abert   aborrec        abr  abr caix  abr cham  \
Topic 1  0.386804  0.350305  0.351014   4.164768  0.469326  0.938233   
Topic 2  3.399994  9.082511  0.612928   0.424923  0.463824  0.338135   
Topic 3  1.218043  4.339285  2.317945  19.032705  2.199225  1.293638   

         abr embal  abr fech  abr reclamaca     abril    absurd  \
Topic 1   0.363355  0.338861       5.910072  1.162842  3.145555   
Topic 2   0.395551  0.366610       0.338187  0.336597  0.441638   
Topic 3   3.625520  1.674883       0.399014  0.920164  9.669532   

         absurd entreg       acab  acab deix  acab fic  acab nao  acab receb  \
Topic 1       1.745037   0.365774   0.334152  0.338827  0.354560    1.764493   
Topic 2       0.371786  32.277051   2.679434  1.896748  3.251918    0.364222   
Topic 3       0.381468   0.403476   0.334295  0.348366  0.676486    0.610012   

         acab ruim     aceit     acend     acert     acess  acessori  \
Topic 1   0.334419  0.345151  0.335625  1.568513  0.568607  2.380859   
Topic 2   3.388207  0.355439  2.563189  0.336404  0.362181  0.619340   
Topic 3   0.337330  6.127380  0.410255  0.838125  2.297121  2.026609   

               ach  ach absurd   ach bem   ach car  ach cord  ach demor  \
Topic 1   0.403554    0.334059  0.334949  0.340274  0.333792   5.305508   
Topic 2  56.616760    0.339738  2.190221  1.242105  1.090878   0.365849   
Topic 3  13.374191    1.449879  0.352237  0.671707  0.333870   0.347912   

          ach dev  ach entreg  ach estranh  ach fornec  ach fret   ach loj  \
Topic 1  0.374385    0.358961     0.338772    0.342467  0.341036  0.336338   
Topic 2  0.388818    1.620309     1.606097    1.214944  0.362505  0.349587   
Topic 3  4.612219    1.019893     0.594224    0.345144  2.264302  2.136414   

         ach mant  ach mater  ach melhor   ach nao  ach pequen  ach pess  \
Topic 1  0.334493   0.336166    0.335768  0.392095    0.338314  0.351218   
Topic 2  0.340318   2.236262    2.060591  0.974248    4.620428  1.750799   
Topic 3  2.116753   0.338901    0.343714  1.262365    0.339636  0.656571   

         ach pouc  ach praz  ach prec  ach produt  ach qual   ach vei  \
Topic 1  0.341279  1.331299  0.334514    0.337801  0.334322  0.352847   
Topic 2  4.194469  0.351394  2.581642    9.196508  2.216051  1.037243   
Topic 3  0.339332  2.293299  0.335461    0.373882  0.335064  1.559901   

             acim     acion       aco  acompanh  acompanh ped  \
Topic 1  2.217915  0.347159  0.390073  4.087561      2.294328   
Topic 2  0.398943  0.382999  1.857151  3.566891      0.336174   
Topic 3  0.358477  2.782816  0.730078  1.948134      0.363052   

         acompanh produt  acondicion    acontec  acontec compr  acontec nao  \
Topic 1         0.378823    0.350372   6.810295       0.347546     0.875072   
Topic 2         0.547035    2.776555   0.355561       0.347457     0.341488   
Topic 3         2.053443    0.357734  11.951580       1.491302     1.183121   

         acontec produt     acord   acostum   acredit  acredit nao     acril  \
Topic 1        0.874218  0.467249  0.344305  0.379175     0.341022  0.335856   
Topic 2        0.349489  5.975097  0.423961  5.307044     1.505265  4.770820   
Topic 3        2.207312  5.019631  2.416852  5.944883     0.415370  0.338058   

         acril fic      acus     adapt     adequ      ader      ades  \
Topic 1   0.334120  2.427847  0.687051  0.365380  0.347051  0.425641   
Topic 2   1.903811  0.362398  4.406852  2.622543  1.502334  6.036624   
Topic 3   0.334144  0.680276  0.516295  2.744016  0.334735  4.323900   

         ades pared  ades pra    adiant      ador    adquir  adquir nao  \
Topic 1    0.801935  0.333924  1.912375  0.336856  4.956330    0.338412   
Topic 2    0.339787  0.351658  0.683843  9.746496  0.436177    0.339295   
Topic 3    1.407882  1.130258  0.426343  0.360493  9.751716    3.156760   

         adquir produt     adult      afet  afet ades    afinal     afirm  \
Topic 1       1.574661  1.115870  0.335327   0.333924  0.946975  0

#### Topics

🎁 We provided you with some functions:
* `topic_word` returns the top words with their weights for one topic
* `print_topics` prints the different topics found by the LDA with their topwords

In [35]:
def topic_word(vectorizer, model, topic, topwords, with_weights = True):
    topwords_indexes = topic.argsort()[:-topwords - 1:-1]
    if with_weights == True:
        topwords = [(vectorizer.get_feature_names_out()[i], round(topic[i],2)) for i in topwords_indexes]
    if with_weights == False:
        topwords = [vectorizer.get_feature_names_out()[i] for i in topwords_indexes]
    return topwords

In [36]:
def print_topics(vectorizer, model, topwords):
    for idx, topic in enumerate(model.components_):
        print("-"*20)
        print("Topic %d:" % (idx))
        print(topic_word(vectorizer, model, topic, topwords))
        

🕵🏻‍♂️ Print the topics with their topwords:

In [ ]:
# YOUR CODE HERE
print_topics(vectorizer, model, topwords)


🇧🇷 A bit of Brazilian Portuguese words here:
- _cadeiras = chairs_
- _producto = product_
- _bom = good_
- _comprei = bought_
- _veio = came_
- _duas = two_
- _nao = not_
- _entregue = delivered_
- _pecas = part_
- _ainda = yet_
- _recebi = received_

👉 Show the top words associated to a topic

In [ ]:
# YOUR CODE HERE
topic_word_mixture = [topic_word(vectorizer, lda, topic, topwords = 5, with_weights = False)
                      for topic in lda.components_]
topic_word_mixture

In [ ]:
df["most_important_words"] = df["most_important_topic"].apply(lambda i: topic_word_mixture[i])

In [ ]:
df[["review_id", 
        "review_score", 
        "product_category_name",
        "full_review_cleaned",
        "most_important_topic",
        "most_important_words"]
      ].head(3)